In [1]:
import more_itertools
from typing import List
from embedding_generator_default import EmbeddingGeneratorDefault
from mongo_db_client import MongoDbClient
from pre_processer_default import PreProcesserDefault
import tensorflow as tf
import numpy as np
from models import PairDbDoc

/Users/beto/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/beto/miniconda3/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/beto/miniconda3/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-06-25 14:51:58.526004: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-25 14:51:58.526106: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
db_client = MongoDbClient()
embedding_generator = EmbeddingGeneratorDefault()
pre_processer = PreProcesserDefault()

In [4]:
def get_embeddings_from_batched_pairs(pairs: List[PairDbDoc]):
  code_sentences = [pre_processer.process_code(pair['code_tokens']) for pair in pairs]
  comment_sentences = [pre_processer.process_text(pair['comment_tokens']) for pair in pairs]

  return embedding_generator.from_code(code_sentences), embedding_generator.from_text(comment_sentences)

After tests using 64, 128 and 256 for batch_size and 1024 pairs from db, the fastest for generate embeddings was 64

In [5]:
pairs = list(db_client.get_pairs_collection().find().limit(64))
code_emb, comment_emb = get_embeddings_from_batched_pairs(pairs)


In [6]:
from keras import Sequential
from keras.layers import Input, Dense, Concatenate, Reshape
from utils import encoder_hidden_size, encoder_seq_len

transformer_embedding_shape = (encoder_seq_len, encoder_hidden_size)
hidden_activation = 'relu'
output_activation = 'sigmoid'

code_input, comment_input = Input(shape=transformer_embedding_shape, name="code_input"), Input(shape=transformer_embedding_shape, name="comment_input")
code_input, comment_input
# code_input = Reshape(target_shape=(-1, ))()
# comment_input = Reshape(target_shape=(-1, ))() 
# merged = Concatenate(name="joint_embedding")([code_input, comment_input])

# merged

(<KerasTensor: shape=(None, 512, 384) dtype=float32 (created by layer 'code_input')>,
 <KerasTensor: shape=(None, 512, 384) dtype=float32 (created by layer 'comment_input')>)

In [7]:
reshaped_code = Reshape(target_shape=(-1, ))(code_input)
reshaped_comment = Reshape(target_shape=(-1, ))(comment_input)
reshaped_code, reshaped_comment

(<KerasTensor: shape=(None, 196608) dtype=float32 (created by layer 'reshape')>,
 <KerasTensor: shape=(None, 196608) dtype=float32 (created by layer 'reshape_1')>)

In [8]:
merged = Concatenate(name="joint_embedding")([reshaped_code, reshaped_comment])
merged

<KerasTensor: shape=(None, 393216) dtype=float32 (created by layer 'joint_embedding')>

In [10]:
tf.type_spec_from_value(['1', '2', '1'])

TensorSpec(shape=(3,), dtype=tf.string, name=None)

In [13]:
tf.ones(shape=(1, ))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>

In [14]:
tf.concat([tf.ones(shape=(1, )), tf.zeros(shape=(1, ))], axis=0)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 0.], dtype=float32)>

In [15]:
code_tokens = ["@","VisibleForTesting","protected","static","boolean","notAllowedStrategy","(","DockerSlaveTemplate","template",")","{","if","(","isNull","(","template",")",")","{","LOG",".","debug","(","\"Skipping DockerProvisioningStrategy because: template is null\"",")",";","return","true",";","}","final","RetentionStrategy","retentionStrategy","=","template",".","getRetentionStrategy","(",")",";","if","(","isNull","(","retentionStrategy",")",")","{","LOG",".","debug","(","\"Skipping DockerProvisioningStrategy because: strategy is null for {}\"",",","template",")",";","}","if","(","retentionStrategy","instanceof","DockerOnceRetentionStrategy",")","{","if","(","template",".","getNumExecutors","(",")","==","1",")","{","LOG",".","debug","(","\"Applying faster provisioning for single executor template {}\"",",","template",")",";","return","false",";","}","else","{","LOG",".","debug","(","\"Skipping DockerProvisioningStrategy because: numExecutors is {} for {}\"",",","template",".","getNumExecutors","(",")",",","template",")",";","return","true",";","}","}","if","(","retentionStrategy","instanceof","RetentionStrategy",".","Demand",")","{","LOG",".","debug","(","\"Applying faster provisioning for Demand strategy for template {}\"",",","template",")",";","return","false",";","}","\/\/ forbid by default","LOG",".","trace","(","\"Skipping YAD provisioning for unknown mix of configuration for {}\"",",","template",")",";","return","true",";","}"]
comment_tokens = ["Exclude","unknown","mix","of","configuration","."]

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.code_input = Input(shape=transformer_embedding_shape, name="code_input") 
    self.comment_input = Input(shape=transformer_embedding_shape, name="comment_input")
    self.reshaped_code = Reshape(target_shape=(-1, ))
    self.reshaped_comment = Reshape(target_shape=(-1, ))
    self.merged = Concatenate(name="joint_embedding")
    self.hidden1 = Dense(400, activation='relu')
    self.hidden2 = Dense(200, activation='relu')
    self.hidden3 = Dense(100, activation='relu')

  def call(self, inputs):
    return 1

In [ ]:
class TransfomerEmbedding(tf.keras.layers.Layer):
  
  def __init__(self, units=32, input_dim=32):
    super().__init__()

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [24]:
dataset = db_client.get_pairs_collection().find().limit(10)
model = MyModel()
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)
model.fit(dataset)

AttributeError: 'ObjectId' object has no attribute 'shape'

In [1]:
def test1():
  for i in range(10):
    yield i, i+1, i+2

def test2():
  for i in range(10):
    yield ((i, i+1), i+2)

In [3]:
for input1, input2, target in test1():
    print(f'i1: {input1} i2: {input2} target: {target}')

i1: 0 i2: 1 target: 2
i1: 1 i2: 2 target: 3
i1: 2 i2: 3 target: 4
i1: 3 i2: 4 target: 5
i1: 4 i2: 5 target: 6
i1: 5 i2: 6 target: 7
i1: 6 i2: 7 target: 8
i1: 7 i2: 8 target: 9
i1: 8 i2: 9 target: 10
i1: 9 i2: 10 target: 11


In [6]:
for item in test2():
    print(f'inputs: {item[0]} target: {item[1]}')

inputs: (0, 1) target: 2
inputs: (1, 2) target: 3
inputs: (2, 3) target: 4
inputs: (3, 4) target: 5
inputs: (4, 5) target: 6
inputs: (5, 6) target: 7
inputs: (6, 7) target: 8
inputs: (7, 8) target: 9
inputs: (8, 9) target: 10
inputs: (9, 10) target: 11


In [3]:
queries = db_client.get_queries_collection()
results = queries.update_many(
  { "language": "Java" },
  {"$set": {"language": "java" } }
)

results.modified_count

339

In [4]:
results = queries.update_many(
  { "language": "Python" },
  {"$set": {"language": "python" } }
)
results

In [6]:
def get_target(target, batch_size):
  target_embedding = tf.convert_to_tensor([target for _ in range(batch_size)])
  return target_embedding

get_target(0, 1).shape

TensorShape([1])

In [7]:
np.full((1, ), 0).shape

(1,)